# Neurons use-case

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load neurons

In [ ]:
assembly_name = 'Neurons'
be.remove_assembly(assembly_name)
neurons_assembly = be.add_assembly(assembly_name)

center = [200, 1500, 395]
# center = [345, 1359, 600]

neurons_model = be.add_neurons(
    assembly_name=assembly_name,
    population_name='ngv',
    geometry_quality=be.GEOMETRY_QUALITY_LOW,
    # population_color_scheme=be.POPULATION_COLOR_SCHEME_ID,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION,
    use_sdf=True,
    radius_multiplier=5.5,
    load_somas=True, load_basal_dendrites=True, load_apical_dendrites=True, load_axons=True,
    generate_internals=True, generate_externals=False,
    # sql_node_filter='guid=115109',
    sql_node_filter = 'sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 200 AND guid %% 10 = 0' % (center[0], center[1], center[2]),
    # sql_section_filter='sqrt(pow(n.x + s.x - %f, 2) + pow(n.y + s.y - %f, 2) + pow(n.z + s.z - %f, 2)) < 200' % (center[0], center[1], center[2]),
)

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, palette_size)

    for material_id in material_ids:
        mid = material_id % palette_size
        color = palette[mid]
        if mid in [be.NEURON_MATERIAL_AFFERENT_SYNPASE, be.NEURON_MATERIAL_EFFERENT_SYNPASE]:
            colors.append(color)
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(1.0)
            user_params.append(1.0)
            specular_exponents.append(3.0)
            emissions.append(0.75)
        elif mid == be.NEURON_MATERIAL_MITOCHONDRION:
            colors.append([1.0, 0.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.5)
            user_params.append(1.0)
            specular_exponents.append(6.0)
            emissions.append(0.25)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(1.0)
            user_params.append(0.2)
            specular_exponents.append(30.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            colors.append(color)
            opacities.append(0.75)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.01)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_STEATH:
            opacities.append(1.0)
            colors.append([0.4, 0.3, 0.5])
            shading_modes.append(be.SHADING_MODE_ELECTRON)
            glossinesses.append(1.0)
            user_params.append(2.0)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        else:
            # Membrane
            colors.append(color)
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.01)
            specular_exponents.append(50.0)
            emissions.append(0.0)
            
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [ ]:
offset = 0
print(be.get_model_ids())
set_morphology_materials(offset, 'Blues', be.NB_MATERIALS_PER_NEURON)
set_morphology_materials(offset + 1, 'Reds_r', be.NB_MATERIALS_PER_ASTROCYTE)
set_morphology_materials(offset + 2, 'Greens', be.NB_MATERIALS_PER_ASTROCYTE)
core.set_renderer()

In [ ]:
status = core.set_renderer(
    current='bio_explorer',
    subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 10.0
params.shadows = 1.0
params.soft_shadows = 1.0
params.epsilon_factor = 500.0
params.max_bounces = 10
status = core.set_renderer_params(params)
status = core.set_renderer()